In [1]:
import re
import pandas as pd
from datetime import datetime
wd = '/Volumes/PEDS/RI Biostatistics Core/Shared/Shared Projects/Vigers/CF/Christine Chan/Prepost Triakfta/'

In [2]:
# Import
chco_dates = pd.read_csv(wd + 'Data_Cleaned/chco_trikafta_dates.csv')
# Fix selected dates
chco_dates.loc[chco_dates.MRN == 1053930,'Earliest Trikafta Date in PortCF'] = "12/1/2019"
chco_dates.loc[chco_dates.MRN == 1695512,'Earliest Trikafta Date in PortCF'] = "12/12/2019"
chco_dates.loc[chco_dates.MRN == 815274,'Earliest Trikafta Date in PortCF'] = "12/13/2019"
chco_dates.loc[chco_dates.MRN == 1078408,'Earliest Trikafta Date in PortCF'] = "3/9/2020"
chco_dates.loc[chco_dates.MRN == 899717,'Earliest Trikafta Date in PortCF'] = "2/14/2020"
# CFRD only
chco_dates = chco_dates.loc[chco_dates['CFRD yes=1'] == 1]
# Remove those without Trikafta date
chco_dates.dropna(subset = ['Earliest Trikafta Date in PortCF'],inplace = True)
# Add glycemic data
chco_a1c = pd.read_csv(wd + 'Data_Cleaned/chco_a1c_ogtt.csv')
chco = pd.merge(chco_dates,chco_a1c,on = ['MRN'],how = 'left')
# Add PFTs
chco_pfts = pd.read_csv(wd + 'Data_Cleaned/chco_bmi_pft.csv')
idx = chco_pfts['MRN'].isin(chco_dates['MRN']) 
chco_pfts = chco_pfts[idx]
chco = pd.merge(chco,chco_pfts,on = ['MRN','Date'],how = 'outer')
# Add CGM
chco_cgm = pd.read_csv(wd + 'Data_Cleaned/chco_cgm.csv')
chco_cgm['CFF ID'] = [int(re.sub('_.*','',s)) for s in chco_cgm['subject_id']]
chco_cgm['Date'] = [re.sub(' .*','',s) for s in chco_cgm['date_cgm_placement']]
chco = pd.merge(chco,chco_cgm,on = ['CFF ID','Date'],how = 'outer')
# Sort
chco.sort_values(by = ['MRN','Date'],inplace = True)
# Fill missing 
fill = ['DOB','CFF ID','Earliest Trikafta Date in PortCF','Sex','Race','Hispanic/Latinx','Genotypes1','Genotypes2','Pancreatic Status','CFRD Diagnosis Date']
chco[fill] = chco.groupby('MRN')[fill].ffill()
chco[fill] = chco.groupby('MRN')[fill].bfill()


In [3]:
chco.to_csv("temp.csv",index=False)